# Time series analysis with HyP3 and MintPy

This notebook walks through performing a time-series analysis of the 2019 Ridgecrest, CA earthquake with On Demand InSAR products from the Alaska Satellite facility and MintPy. We'll:

1. Use the [ASF Search Python package](https://docs.asf.alaska.edu/asf_search/basics/) to:
   - Search ASF's catalog for Sentinel-1 SAR products covering the [Ridgecrest earthquake](https://earthquake.usgs.gov/storymap/index-ridgecrest.html)
   - Select a reference scene to generate a baseline stack
   - Select a [short baseline subset (SBAS)](https://docs.asf.alaska.edu/vertex/sbas/) of scene pairs for InSAR processing


2. Use the [HyP3 Python SDK](https://hyp3-docs.asf.alaska.edu/using/sdk/) to:
   - Request On Demand InSAR products from ASF HyP3
   - Download the InSAR products when they are done processing


3. Use [GDAL](https://gdal.org/api/index.html#python-api) and [MintPy](https://mintpy.readthedocs.io/en/latest/) to:
   - Prepare the InSAR products for MintPy
   - perform a time-series analysis with MintPy
   
---

**Note:** This notebook does assume you have some familiarity with InSAR processing with MintPy already, and is a minimal example without much context or explanations. If you're new to InSAR and MintPy, I suggest checking out:
* our [InSAR on Demand Story Map](https://storymaps.arcgis.com/stories/68a8a3253900411185ae9eb6bb5283d3)


* [OpenSARlab's](https://opensarlab-docs.asf.alaska.edu/) highly detailed walkthrough of using HyP3 + MintPy via these notebooks:
  * [Prepare a HyP3 InSAR Stack for MintPy](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb)
  * [MintPy Time-series Analysis](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/MintPy_Time_Series_From_Prepared_Data_Stack.ipynb)
  
    Note: While these notebooks make some assumptions you're working in OpenSARlab, you can run these 
    notebooks outside OpenSARlab by creating [this conda environment](https://github.com/ASFOpenSARlab/opensarlab-envs/blob/main/Environment_Configs/insar_analysis_env.yml).

## 0. Initial Setup

To run this notebook, you'll need a conda environment with the required dependencies. You can set up a new environment (recommended) and run the jupyter server like:
```shell
conda create -n hyp3-mintpy python>=3.10 asf_search hyp3_sdk "mintpy>=1.5.2" pandas jupyter ipympl

conda activate hyp3-mintpy
jupyter notebook hyp3_insar_stack_for_ts_analysis.ipynb
```
Or, install these dependencies into your own environment:
```shell
conda install hyp3-mintpy python>=3.10 asf_search hyp3_sdk "mintpy>=1.5.2" pandas jupyter ipympl

jupyter notebook hyp3_insar_stack_for_ts_analysis.ipynb
```

### Set parameters

In [ ]:
from pathlib import Path
from dateutil.parser import parse as parse_date

project_name = '2019_ridgecrest_stac'
work_dir = Path.cwd() / project_name
stac_dir = work_dir / 'stac'

stack_start = parse_date('2019-06-10 00:00:00Z')
stack_end = parse_date('2019-07-21 00:00:00Z')
max_temporal_baseline = 13  # days

mintpy_dir.mkdir(parents=True, exist_ok=True)

## 1. Select InSAR pairs with ASF Search

In [ ]:
import asf_search as asf
import pandas as pd

search_results = asf.geo_search(
    platform=asf.PLATFORM.SENTINEL1,
    intersectsWith='POINT(-117.599330 35.769500)',
    start='2019-06-10',
    end='2019-07-21',
    processingLevel=asf.PRODUCT_TYPE.SLC,
    beamMode=asf.BEAMMODE.IW,
    flightDirection=asf.FLIGHT_DIRECTION.ASCENDING,
)

In [ ]:
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry']
data = [list(scene.properties.values()) + [scene.geometry] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

In [ ]:
sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference) & (stack.temporalBaseline - rt <= max_temporal_baseline) & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

## 2. Request On Demand InSAR products from ASF HyP3

Use your [NASA Earthdata login](https://urs.earthdata.nasa.gov/) to connect to [ASF HyP3](https://hyp3-docs.asf.alaska.edu/).

In [ ]:
import hyp3_sdk as sdk


hyp3 = sdk.HyP3()

In [ ]:
jobs = sdk.Batch()
for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=project_name, include_dem=True, include_look_vectors=True)

In [ ]:
jobs = hyp3.watch(jobs)

In [ ]:
jobs = hyp3.find_jobs(name=project_name, user_id='ffwilliams2')

In [ ]:
from hyp3_sdk import stac

stac.create_stac_collection(jobs, stac_dir)

## 3. View Data

In [ ]:
import pystac
from hyp3_sdk import load


collection = pystac.Collection.from_file(stac_dir / 'collection.json')
items = list(collection.get_all_items())
dataset = load.create_xarray_dataset(items)

In [ ]:
dataset

In [ ]:
eq_ds = dataset[dataset.coords['id'] == 'S1BB_20190628T014958_20190710T014959_VVP012_INT80_G_ueF_006C'].sel(band='unw_phase')
eq_ds = eq_ds.load()

In [ ]:
import leafmap


m = leafmap.Map()
unw_image = leafmap.array_to_image(eq_ds.to_numpy(), cellsize=eq_ds.transform[0], crs=eq_ds.crs, transform=eq_ds.transform)
m.add_raster(unw_image, colormap='coolwarm', layer_name='Unwrap')
m

In [ ]:
from pyproj.transformer import Transformer
from shapely.geometry import Polygon, shape


def get_bbox(map, epsg):
    features = m.draw_features
    if len(features) > 1:
        raise ValueError('Only 1 feature can be used. Remove all features from map and start again.')
    polygon = Polygon(shape(features[0]['geometry']))
    minx, miny, maxx, maxy = polygon.bounds
    transformer = Transformer.from_crs('EPSG:4326', f'EPSG:{epsg}', always_xy=True)
    (minx, maxx), (miny, maxy) = transformer.transform([minx, maxx], [miny, maxy])
    return [round(coord) for coord in (minx, maxx, miny, maxy)]

In [ ]:
get_bbox(m, dataset.epsg.item())

## 4. Run MintPy time series analysis

In [ ]:
%%time
from hyp3_sdk import load

load.create_mintpy_inputs(stac_dir / 'collection.json', subset_geo=get_bbox(m, dataset.epsg.item()), mintpy_dir=mintpy_dir)

In [ ]:
mintpy_config = mintpy_dir / 'mintpy_config.cfg'
mintpy_config.write_text(
    f"""
mintpy.load.processor = hyp3
##---------misc:
mintpy.plot = no
mintpy.network.coherenceBased = no
mintpy.troposphericDelay.method = no
mintpy.reference.date = 20190610
"""
)

In [ ]:
!smallbaselineApp.py --dir {mintpy_dir} --start modify_network {mintpy_config}

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from mintpy.cli import view, tsview

In [ ]:
# view.main([f'{mintpy_dir}/velocity.h5'])

In [ ]:
tsview.main([f'{mintpy_dir}/timeseries.h5'])

## 5. Comparsion

In [ ]:
!du -shm 2019_ridgecrest

In [ ]:
!du -shm 2019_ridgecrest/data

In [ ]:
3842 - 3622

In [ ]:
!du -shm 2019_ridgecrest_stac

In [ ]:
data_percent = 260 / 3842
f'The STAC method produces the same result, but creates only {data_percent:.1%} of the data!'

In [ ]:
round((3842 / 11) * 250 / 1000, 1)

In [ ]:
round((260 / 11) * 250 / 1000, 1)